In [1]:
import numpy as np
import pandas as pd
from multiprocessing import Pool, cpu_count
import gc
import time
gc.enable()
# import seaborn as sns
# import matplotlib.pyplot as plt
import warnings
import re
import requests
# import folium
# import branca.colormap as cm
# import geopy
from tqdm import tqdm_notebook as tqdm
import json
import os
# import geojson

warnings.filterwarnings('ignore')

In [2]:
%%time
# Load from preprocessed data
sd = pd.read_feather('../data/Final_Divvy_data_2018.feather')

CPU times: user 106 ms, sys: 326 ms, total: 432 ms
Wall time: 724 ms


In [3]:
sd.shape[0]

216478

In [4]:
def get_data_of_a_day(month, day):
    sub_df = sd[(sd['month'] == month) & (sd['day'] == day)]
    lat_arr = np.array(sub_df.lat_ave)
    lon_arr = np.array(sub_df.lon_ave)
    flg_arr = np.array(sub_df.total_in > sub_df.dp_max)
    return lat_arr, lon_arr, flg_arr

In [5]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from ipywidgets import widgets
import plotly.graph_objs as go
from plotly import tools

init_notebook_mode(connected=True)

In [6]:
## Global setup for plotting
# API key
'pk.eyJ1IjoibWVydnluMTUyIiwiYSI6ImNqeW92MzBqYjAycHUzZ28ycmQzeDlueGQifQ.l-Qt__CZAv-nnuUC0ZxiZA'
mapbox_access_token = 'pk.eyJ1IjoibWVydnluMTUyIiwiYSI6ImNqeW92MzBqYjAycHUzZ28ycmQzeDlueGQifQ.l-Qt__CZAv-nnuUC0ZxiZA'

In [7]:
## Function to get map data
def _get_map_data(df, month, day, nm, hottag=True, color='orange', subplot='mapbox', 
                  op=0.5, sz=15, vis=False, sl=True):
    if hottag:
        sub_df = df[(df['month'] == month) & (df['day'] == day) & (df.total_in >= df.dp_max)]
        print("Number of hot stations: ", sub_df.shape[0])
    else:
        sub_df = df[(df['month'] == month) & (df['day'] == day) & (df.total_in < df.dp_max)]
        print("Number of inactive stations: ", sub_df.shape[0])
        
    return go.Scattermapbox(
        lat=list(np.array(sub_df.lat_ave)),
        lon=list(np.array(sub_df.lon_ave)),
#         text=["<b>Station id</b>: {}".format(
#             int(sub_df.id),
#         )],
        mode='markers',
        hoverinfo = 'text',
        marker=go.scattermapbox.Marker(
            size=sz,
            color=color,
            opacity=op
        ),   
        subplot=subplot,
        visible=vis,
        name=nm,
        showlegend=sl,
    )

In [8]:
## Style of mixed plot
layout = {
    'title': {
        'text': 'Daily Demand for Each Station',
        'font': dict(
            family='Open Sans, sans-serif',
            size=30,
            color='Black'
        ),
    },
    'mapbox': go.layout.Mapbox(
        accesstoken=mapbox_access_token,
        bearing=45,
        domain={'x': [0.32, 1], 'y': [0, 1]},
        center=go.layout.mapbox.Center(
            lat=41.879, 
            lon=-87.636
        ),
        pitch=60,
        zoom=15.2,
        style = 'mapbox://styles/mervyn152/cjy0sm1q506il1cp9gt19y457',
#         style = "mapbox://styles/mervyn152/cjy2i8m1y1s5s1cnxcto3gppa",
    ),  
    'mapbox2': go.layout.Mapbox(
        accesstoken=mapbox_access_token,
        bearing=0,
        domain={'x': [0, 0.28], 'y': [0, 1]},
        center=go.layout.mapbox.Center(
            lat=41.898, 
            lon=-87.67
        ),
        pitch=0,
        zoom=9,
        style = 'dark'
    ), 
#     'showlegend': True,
    'autosize': True,
    'legend': dict(
        font=dict(
                family='Open Sans, sans-serif',
                size=18,
                color="white"
            ),
        orientation="h", 
        x=1, 
        y=0,
        xanchor="right",
        yanchor="bottom",
        bgcolor="Black",
    ),
    'margin': go.layout.Margin(l=10, r=10, b=30, t=50, pad=6)
}

In [9]:
## Get data
data = []

sun_m = 7
sun_d = 1
# data.append(_get_map_data(sd, 7, 1, 'Low Demand', hottag=False, color='orange', subplot='mapbox', 
#                           op=1, sz=25, vis=True, sl=True))
# data.append(_get_map_data(sd, 7, 1, 'Heavy Demand', hottag=True, color='red', subplot='mapbox', 
#                           op=1, sz=25, vis=True, sl=True))
# data.append(_get_map_data(sd, 7, 1, 'Low Demand', hottag=False, color='orange', subplot='mapbox2', 
#                           op=0.9, sz=12, vis=True, sl=False))
# data.append(_get_map_data(sd, 7, 1, 'Heavy Demand', hottag=True, color='red', subplot='mapbox2', 
#                           op=0.9, sz=12, vis=True, sl=False))
data.append(_get_map_data(sd, sun_m, sun_d, 'Low Demand', hottag=False, color='yellow', subplot='mapbox', 
                          op=0.8, sz=25, vis=False, sl=True))
data.append(_get_map_data(sd, sun_m, sun_d, 'Heavy Demand', hottag=True, color='red', subplot='mapbox', 
                          op=0.6, sz=25, vis=False, sl=True))
data.append(_get_map_data(sd, sun_m, sun_d, 'Low Demand', hottag=False, color='yellow', subplot='mapbox2', 
                          op=0.8, sz=12, vis=False, sl=False))
data.append(_get_map_data(sd, sun_m, sun_d, 'Heavy Demand', hottag=True, color='red', subplot='mapbox2', 
                          op=0.6, sz=12, vis=False, sl=False))
data.append(_get_map_data(sd, 10, 29, 'Low Demand', hottag=False, color='yellow', subplot='mapbox', 
                          op=0.8, sz=25, vis=True, sl=True))
data.append(_get_map_data(sd, 10, 29, 'Heavy Demand', hottag=True, color='red', subplot='mapbox', 
                          op=0.6, sz=25, vis=True, sl=True))
data.append(_get_map_data(sd, 10, 29, 'Low Demand', hottag=False, color='yellow', subplot='mapbox2', 
                          op=0.8, sz=12, vis=True, sl=False))
data.append(_get_map_data(sd, 10, 29, 'Heavy Demand', hottag=True, color='red', subplot='mapbox2', 
                          op=0.6, sz=12, vis=True, sl=False))

Number of inactive stations:  385
Number of hot stations:  202
Number of inactive stations:  385
Number of hot stations:  202
Number of inactive stations:  401
Number of hot stations:  209
Number of inactive stations:  401
Number of hot stations:  209


In [10]:
sun_annotations = [dict(x=0.27,
                        y=0.92,
                        xanchor='right',
                        xref="paper", yref="paper",
                        text="<br>Heavy: 385<br>&nbsp;&nbsp;&nbsp;Low: 202",
                        font=dict(color='white', size=18, family='Open Sans, sans-serif', ),
                        ax=0, ay=0),
                   dict(x=0.32,
                        y=1,
                        xanchor='left',
                        yanchor='top',
                        xref="paper", yref="paper",
                        text="Sunday",
                        bgcolor="black",
                        borderpad=8,
                        font=dict(color='white', size=25, family='Open Sans, sans-serif', ),
                        ax=0, ay=0),
                  ]
mon_annotations = [dict(x=0.27,
                        y=0.92,
                        xanchor='right',
                        xref="paper", yref="paper",
                        text="Heavy: 401<br>&nbsp;&nbsp;&nbsp;Low: 209" ,
                        font=dict(color='white', size=18, family='Open Sans, sans-serif', ),
                        ax=0, ay=0),
                   dict(x=0.32,
                        y=1,
                        xanchor='left',
                        yanchor='top',
                        xref="paper", yref="paper",
                        text="Monday",
                        bgcolor="black",
                        borderpad=8,
                        font=dict(color='white', size=25, family='Open Sans, sans-serif', ),
                        ax=0, ay=0),
                  ]
# mon_annotations = [dict(x="2015-05-01",
#                         y=df.Low.mean(),
#                         xref="x", yref="y",
#                         text="Low Average:<br> %.2f" % df.Low.mean(),
#                         ax=-40, ay=40),
#                    dict(x=df.High.idxmin(),
#                         y=df.Low.min(),
#                         xref="x", yref="y",
#                         text="Low Min:<br> %.2f" % df.Low.min(),
#                         ax=0, ay=40)]

updatemenus = list([
    dict(type="buttons",
         active=0,
         buttons=list([
            dict(label = 'Sep 16th, 2018',
                 method = 'update',
                 args = [
                     {'visible': [True, True, True, True, False, False, False, False]},
                     {"annotations": sun_annotations}
                     ]),
            dict(label = 'Oct 29th, 2018',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, True, True, True, True, ]},
                         {"annotations": mon_annotations},
                        ]),

        ]),
        direction = 'left',
        showactive = True,
        pad = {'r': 0, 't': 0},
        x = 0,
        xanchor = 'left',
        y = -0.1,
        yanchor = 'bottom',
#         bgcolor = 'lightgrey',
#         bordercolor = 'white',
        font = dict(size=16, color='#101010', family='Open Sans, sans-serif', )
    ),
])

layout['updatemenus'] = updatemenus

In [11]:
figure = {}
figure['data'] = data
figure['layout'] = layout


SHOW = True
SAVE = True

if SHOW:
    iplot(figure, config={'displayModeBar': False})

if SAVE:
    plot(figure, config={'displayModeBar': False}, filename="example_usage_2.html")

In [341]:
map_from = folium.Map(location = [41.90, -87.64], zoom_start = 11,tiles=None)
folium.TileLayer(tiles='cartodbpositron', overlay=True, max_zoom=13, min_zoom=10.5).add_to(map_from)
d1_group = folium.FeatureGroup(name="July 1st, 2018 (Sun)", overlay=False)
d2_group = folium.FeatureGroup(name="July 2nd, 2018 (Mon)", overlay=False)
d3_group = folium.FeatureGroup(name="Oct 28th, 2018 (Sun)", overlay=False)
d4_group = folium.FeatureGroup(name="Oct 29th, 2018 (Mon)", overlay=False)


lat_lst_d1, long_lst_d1, flag_d1 = get_data_of_a_day(7, 1)
lat_lst_d2, long_lst_d2, flag_d2 = get_data_of_a_day(7, 2)
lat_lst_d3, long_lst_d3, flag_d3 = get_data_of_a_day(10, 28)
lat_lst_d4, long_lst_d4, flag_d4 = get_data_of_a_day(10, 29)

rd = 8
for lat, long, f in zip(lat_lst_d1, long_lst_d1, flag_d1):
    if f:
        folium.CircleMarker([lat, long], color="red", fill=True, radius=rd, weight=0, fill_opacity=0.5).add_to(d1_group)
    else:
        folium.CircleMarker([lat, long], color="orange", fill=True, radius=rd, weight=0, fill_opacity=0.3).add_to(d1_group)
    
for lat, long, f in zip(lat_lst_d2, long_lst_d2, flag_d2):
    if f:
        folium.CircleMarker([lat, long], color="red", fill=True, radius=rd, weight=0, fill_opacity=0.5).add_to(d2_group)
    else:
        folium.CircleMarker([lat, long], color="orange", fill=True, radius=rd, weight=0, fill_opacity=0.3).add_to(d2_group)    

for lat, long, f in zip(lat_lst_d3, long_lst_d3, flag_d3):
    if f:
        folium.CircleMarker([lat, long], color="red", fill=True, radius=rd, weight=0, fill_opacity=0.5).add_to(d3_group)
    else:
        folium.CircleMarker([lat, long], color="orange", fill=True, radius=rd, weight=0, fill_opacity=0.3).add_to(d3_group) 

for lat, long, f in zip(lat_lst_d4, long_lst_d4, flag_d4):
    if f:
        folium.CircleMarker([lat, long], color="red", fill=True, radius=rd, weight=0, fill_opacity=0.5).add_to(d4_group)
    else:
        folium.CircleMarker([lat, long], color="orange", fill=True, radius=rd, weight=0, fill_opacity=0.3).add_to(d4_group) 


d1_group.add_to(map_from)
# d2_group.add_to(map_from)
# d3_group.add_to(map_from)
d4_group.add_to(map_from)


folium.LayerControl(collapsed=False).add_to(map_from)

SAVE = False

if SAVE:
    map_from.save('example_usage.html')
map_from

NameError: name 'folium' is not defined

In [5]:
emtpy_m = folium.Map(location = [41.90, -87.64], zoom_start = 12,tiles=None)
folium.TileLayer(tiles='Stamen Watercolor', overlay=True).add_to(emtpy_m)

In [6]:
SHOW = True

if SHOW:
    emtpy_m
else:
    emtpy_m.save('empty_map.html')

In [7]:
emtpy_m